# Enunciado


## Projeto - Extração de Dados I
------------------------------
## Sistema de Monitoramento de Avanços no Campo da Genômica  

## Contexto:  
O grupo trabalha no time de engenharia de dados na HealthGen, uma empresa especializada em genômica e pesquisa de medicina personalizada. A genômica é o estudo do conjunto completo de genes de um organismo, desempenha um papel fundamental na medicina personalizada e na pesquisa biomédica. Permite a análise do DNA para identificar variantes genéticas e mutações associadas a doenças e facilita a personalização de tratamentos com base nas características genéticas individuais dos pacientes.

A empresa precisa se manter atualizada sobre os avanços mais recentes na genômica, identificar oportunidades para pesquisa e desenvolvimento de tratamentos personalizados e acompanhar as tendências em genômica que podem influenciar estratégias de pesquisa e desenvolvimento. Pensando nisso, o time de dados apresentou uma proposta de desenvolvimento de um sistema que coleta, analisa e apresenta as últimas notícias relacionadas à genômica e à medicina personalizada, e também estuda o avanço do campo nos últimos anos.

O time de engenharia de dados tem como objetivo desenvolver e garantir um pipeline de dados confiável e estável. As principais atividades são:

### 1. Consumo de dados com a News API:  
Implementar um mecanismo para consumir dados de notícias de fontes confiáveis e especializadas em genômica e medicina personalizada, a partir da News API:  
https://newsapi.org/

### 2. Definir Critérios de Relevância:  
Desenvolver critérios precisos de relevância para filtrar as notícias. Por exemplo, o time pode se concentrar em notícias que mencionem avanços em sequenciamento de DNA, terapias genéticas personalizadas ou descobertas relacionadas a doenças genéticas específicas.

### 3. Cargas em Batches:  
Armazenar as notícias relevantes em um formato estruturado e facilmente acessível para consultas e análises posteriores. Essa carga deve acontecer 1 vez por hora. Se as notícias extraídas já tiverem sidos armazenadas na carga anterior, o processo deve ignorar e não armazenar as notícias novamente, os dados carregados não podem ficar duplicados.  
![Alt text](image.png)

### 4. Dados transformados para consulta do público final  
A partir dos dados carregados, aplicar as seguintes transformações e armazenar o resultado final para a consulta do público final:  

4.1 - Quantidade de notícias por ano, mês e dia de publicação;  

4.2 - Quantidade de notícias por fonte e autor;  

4.3 - Quantidade de aparições de 3 palavras chaves por ano, mês e dia de publicação (as 3 palavras chaves serão as mesmas usadas para fazer os filtros de relevância do item 2 (2. Definir Critérios de Relevância)).  

Atualizar os dados transformados 1 vez por dia.  

![Alt text](image-1.png)

----------------------------------------

Além das atividades principais, existe a necessidade de busca de dados por eventos em tempo real quando é necessário, para isso foi desenhado duas opções:

### Opção 1 - Apache Kafka e Spark Streaming:  

Preparar um pipeline com Apache Kafka e Spark Streaming para receber os dados do Produtor Kafka representado por um evento manual e consumir os dados com o Spark Streaming armazenando os resultados temporariamente. Em um processo paralelo, verificar os resultados armazenados temporiamente e armazenar no mesmo destino do item 3 (3. Cargas em Batches) aqueles resultados que ainda não foram armazenados no destino (os dados carregados não podem ficar duplicados). E por fim, eliminar os dados temporários após a verificação e a eventual carga.

![Alt text](image-2.png)

### Opção 2 - Webhooks com notificações por eventos:  
Configurar um webhook para adquirir as últimas notícias a partir de um evento representado por uma requisição POST e fazer a chamada da API e por fim armazenar os resultados temporariamente. Em um processo paralelo, verificar os resultados armazenados temporiamente e armazenar no mesmo destino do item 3 (3. Cargas em Batches) aqueles resultados que ainda não foram armazenados no destino (os dados carregados não podem ficar duplicados). E por fim, eliminar os dados temporários após a verificação e a eventual carga.

![Alt text](image-3.png)

Atividades que precisam ser realizadas pelo grupo definido em aula.  

O grupo precisa construir o pipeline de dados seguindo os requisitos das atividades principais e escolher entre a Opção 1 e Opção 2 para desenvolvimento.  

# Resolução

## 1 Bibliotecas
Para instalar as bibliotecas necessárias do projeto:

In [ ]:
#pip install requests newsapi-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Para importas as bibliotecas e definir as variáveis necessárias:

In [28]:
from newsapi import NewsApiClient
import requests
import time
import pandas as pd
from datetime import datetime
import re

API_KEY = '9a77398581d74beebbd29dbebd159a53'
PALAVRAS_CHAVES_GERAIS = 'genômica OR genômico OR genética OR dna'
PALAVRAS_CHAVES_ESPECIFICAS = ['sequenciamento', 'terapia', 'doença']

## 2 Definições de Funções

Abaixo está localizado todas as definições de funções que será usado no projeto:

In [55]:
# 1. Consumo de dados com a News API -------------------------------------------------------------------------------------------------

## 1 Usando Biblioteca NewsAPI
def fazer_a_request_1( API_KEY:str='9a77398581d74beebbd29dbebd159a53' , PALAVRAS_CHAVES:str='genômica' ):
    # A função realiza o request com API_KEY e PALAVRAS_CHAVES declaradas e é retornado um json
    newsapi = NewsApiClient(api_key=API_KEY)
    response_lib = newsapi.get_everything(q=PALAVRAS_CHAVES,
                                        language='pt',
                                        sort_by='publishedAt'
    )
    return response_lib

## 2 Usando requests
def fazer_a_request_2( API_KEY:str='9a77398581d74beebbd29dbebd159a53' , PALAVRAS_CHAVES:str='genômica' ):
    # A função realiza o request com API_KEY e PALAVRAS_CHAVES declaradas e é retornado um json
    url = f'https://newsapi.org/v2/everything?q={PALAVRAS_CHAVES}&language=pt&sortBy=publishedAt&apiKey={API_KEY}'
    response_requests = requests.get(url).json()
    return response_requests



# 2. Definir Critérios de Relevância -------------------------------------------------------------------------------------------------

def tratar_dados(json_de_noticias, palavras_chave_filtragem) -> pd.DataFrame:
    df = pd.json_normalize(json_de_noticias['articles'])
    df['publishedAt'] = pd.to_datetime(df['publishedAt']).dt.tz_localize(None) # inserido, pq linha abaixo estava dando erro "TypeError: Cannot use .astype to convert from timezone-aware dtype to timezone-naive dtype"
    df['publishedAt'] = df['publishedAt'].astype('datetime64[ms]')
    df.fillna("desconhecido", inplace=True)
    
    # Criar um padrão de filtragem válido a partir da lista de palavras-chave (evita erros com caracteres especiais)
    padrao_filtragem = '|'.join(map(re.escape, palavras_chave_filtragem))

    # Filtrar os resultados com base nas palavras-chave de filtragem
    df_filtrado = df[df['description'].str.contains(padrao_filtragem, case=False)]

    return df_filtrado



# 3. Cargas em Batches: ---------------------------------------------------------------------------------------------------------------

def armazenar_noticias(df) -> None:
    # Recebe o dataframe para ser armazenado, verificar se há alguma notifica repetida e descarta, por fim armazena.
    pass

def carrega_armazenamento():
    # Abre o armazenamentos dos dados coletados e os retonar em dataframe
    pass



# 4. Dados transformados para consulta do público final ------------------------------------------------------------------------------

def qtd_noticia_ano_mes_dia(df_inteiro):
    pass

def qtd_noticia_fonte_autor(df_inteiro):
    pass

def qtd_aparicao_palavras_chaves(df_inteiro):
    pass

def armazenar_dados_estatisticos(A,B,C):
    pass

## 3 Teste

In [53]:
# Instanciar objeto
response = fazer_a_request_1()
# Exibir chaves do dicionário
print(f'Chaves retornadas: {response.keys()}')
# Valores do dicionário
print(f"Status: {response['status']}")
print(f"Total de resultados: {response['totalResults']}")
print(f"Exemplo de artigo: {response['articles'][0]}")

Chaves retornadas: dict_keys(['status', 'totalResults', 'articles'])
Status: ok
Total de resultados: 18
Exemplo de artigo: {'source': {'id': None, 'name': 'Terra.com.br'}, 'author': 'Ansa', 'title': 'Variante Pirola da Covid-19 não é mais perigosa, diz estudo ítalo-brasileiro', 'description': 'O primeiro exemplar da variante Pirola da Covid-19 não tem características que levantam grandes ...', 'url': 'https://www.terra.com.br/byte/ciencia/variante-pirola-da-covid-19-nao-e-mais-perigosa-diz-estudo-italo-brasileiro,69e814ba7c1a78074c8055d4d33f0717goq5sbbc.html', 'urlToImage': 'https://s1.trrsf.com/fe/zaz-mod-t360-icons/svg/logos/terra-16x9-borda.png', 'publishedAt': '2023-09-27T17:49:03Z', 'content': 'O primeiro exemplar da variante Pirola da Covid-19 não tem características que levantam grandes preocupações. Essa foi a conclusão de uma análise genética sobre a cepa (BA.2.86.1). \r\nO trabalho, que … [+1758 chars]'}


In [57]:
# prova real que ambos os metodos de request dão o mesmo resultado
print(fazer_a_request_1() == fazer_a_request_2())
df = tratar_dados(response, PALAVRAS_CHAVES_ESPECIFICAS)
df

True


,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
0,Ansa,Variante Pirola da Covid-19 não é mais perigos...,O primeiro exemplar da variante Pirola da Covi...,https://www.terra.com.br/byte/ciencia/variante...,https://s1.trrsf.com/fe/zaz-mod-t360-icons/svg...,2023-09-27 17:49:03,O primeiro exemplar da variante Pirola da Covi...,desconhecido,Terra.com.br
1,Ansa,"Variante Pirola não é mais perigosa, diz estud...",O primeiro exemplar da variante Pirola da Covi...,https://www.terra.com.br/byte/ciencia/variante...,https://s1.trrsf.com/fe/zaz-mod-t360-icons/svg...,2023-09-27 17:46:03,O primeiro exemplar da variante Pirola da Covi...,desconhecido,Terra.com.br


## 4 Algoritmo

In [6]:
from newsapi import NewsApiClient
import requests
import time

API_KEY = '9a77398581d74beebbd29dbebd159a53'
PALAVRAS_CHAVES_GERAIS = 'genômica OR genômico OR genética OR dna'
INTERVALO = 3600

while True:
    i = 0

    # 1. Consumo de dados com a News API
    response = fazer_a_request_1(API_KEY, PALAVRAS_CHAVES_GERAIS)

    # 2. Definir Critérios de Relevância
    PALAVRAS_CHAVES_ESPECIFICAS = ['sequenciamento', 'terapia', 'doença']
    df_agora = tratar_dados( response , PALAVRAS_CHAVES_ESPECIFICAS ) #em andamento

    # 3. Cargas em Batches:
    armazenar_noticias( df_agora ) # TODO
    
    # 4. Dados transformados para consulta do público final
    if i==23:
        df_inteiro = carrega_armazenamento() # TODO
        a = qtd_noticia_ano_mes_dia( df_inteiro ) # TODO
        b = qtd_noticia_fonte_autor( df_inteiro ) # TODO
        c = qtd_aparicao_palavras_chaves( df_inteiro ) # TODO
        armazenar_dados_estatisticos( a , b , c ) # TODO
        i = 0
    else:
        i += 1
    time.sleep(INTERVALO)
